In [1]:
from selenium import webdriver
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import  WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd 
import time
import os, requests 

In [ ]:
def scrape(path):

    # to avoid windows to close again and again we make use of headless
    options = ChromeOptions()
    options.headless=True
    # make use of chrome for scraping
    service = Service(ChromeDriverManager().install())
    # create a driver using chrome
    driver = Chrome(service=service, options=options)
    # run the driver through url
    driver.get(path)
    driver.maximize_window()
    time.sleep(5)
    
    results = []
    # details = {}
    # get the number of main_card 
    try:
        main_card = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="table-container yf-1jecxey"]')))
        main_table =WebDriverWait(main_card, 10).until(EC.presence_of_element_located((By.XPATH, './/table[@class="table yf-1jecxey noDl"]'))) 
        if main_table:
            th_main = WebDriverWait(main_table, 10).until(EC.presence_of_element_located((By.XPATH, './/thead[@class="yf-1jecxey"]'))) 
            # get the headers as label
            trs =WebDriverWait(th_main, 10).until(EC.presence_of_element_located((By.XPATH, './/tr[@class="yf-1jecxey"]')))
            labels =[th.text.strip() for th in trs.find_elements(By.XPATH, './/th[@class="yf-1jecxey"]')]
            
            tbody = WebDriverWait(main_table, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'tbody')))
            body_trs = WebDriverWait(tbody, 10).until(EC.presence_of_all_elements_located((By.XPATH, './/tr[@class="yf-1jecxey"]')))
            # for tr in trs:
            for tr in body_trs:
                values = [td.text.strip() for td in tr.find_elements(By.XPATH, './/td[@class="yf-1jecxey"]')]
                if values:
                    results.append(dict(zip(labels, values)))
                
    except Exception as e:
        print(f"no main_card found and error is {e}")
        # results.append(details)
    
    
    
    
    
    df = pd.DataFrame.from_dict(results)
    df.to_csv("BTC_csv_10_years.csv", index=False)
    
    # time.sleep(5)
    driver.quit()
    
path = "https://finance.yahoo.com/quote/BTC-USD/history/?period1=1420070400&period2=1740053994"
scrape(path)    

In [29]:
df = pd.read_csv("BTC_csv_10_years.csv")
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,"Feb 20, 2025","96,628.40","97,517.67","96,481.41","97,390.05","97,390.05","27,969,046,528"
1,"Feb 19, 2025","95,532.53","96,855.59","95,011.97","96,635.61","96,635.61","28,990,872,862"
2,"Feb 18, 2025","95,773.81","96,695.38","93,388.84","95,539.55","95,539.55","37,325,720,482"
3,"Feb 17, 2025","96,179.01","97,032.23","95,243.55","95,773.38","95,773.38","27,336,550,690"
4,"Feb 16, 2025","97,580.49","97,725.59","96,060.98","96,175.03","96,175.03","16,536,755,396"
...,...,...,...,...,...,...,...
3699,"Jan 5, 2015",265.08,278.34,265.08,274.47,274.47,"43,962,800"
3700,"Jan 4, 2015",281.15,287.23,257.61,264.20,264.20,"55,629,100"
3701,"Jan 3, 2015",314.85,315.15,281.08,281.08,281.08,"33,054,400"
3702,"Jan 2, 2015",314.08,315.84,313.57,315.03,315.03,"7,860,650"
